Data exploration and analysis
===================

Here we are going to explore the SBB dataset _is-daten-sbb_. We are going to select the features that can be of use for the rest of the project, clean the dataset if it needs to be and then compute some statistics on it.

## Imports

In [28]:
%matplotlib inline
import pandas as pd
print('pandas: {}'.format(pd.__version__))
import numpy as np
print('numpy: {}'.format(np.__version__))
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
import geopy as geo
print('geopy: {}'.format(geo.__version__))


import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import powerlaw
from geopy import distance

pandas: 0.24.2
numpy: 1.16.4
seaborn: 0.9.0
geopy: 1.20.0


## Extraction

In [3]:
filepath='./data/ist-daten-sbb.csv'
df=pd.read_csv(filepath,delimiter=';')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61716 entries, 0 to 61715
Data columns (total 26 columns):
BETRIEBSTAG            61716 non-null object
FAHRT_BEZEICHNER       61716 non-null object
BETREIBER_ID           61716 non-null object
BETREIBER_ABK          61716 non-null object
BETREIBER_NAME         61716 non-null object
PRODUKT_ID             61636 non-null object
LINIEN_ID              61716 non-null int64
LINIEN_TEXT            61716 non-null object
UMLAUF_ID              0 non-null float64
VERKEHRSMITTEL_TEXT    61716 non-null object
ZUSATZFAHRT_TF         61716 non-null bool
FAELLT_AUS_TF          61716 non-null bool
BPUIC                  61716 non-null int64
HALTESTELLEN_NAME      61716 non-null object
ANKUNFTSZEIT           56478 non-null object
AN_PROGNOSE            52344 non-null object
AN_PROGNOSE_STATUS     61716 non-null object
ABFAHRTSZEIT           56477 non-null object
AB_PROGNOSE            52343 non-null object
AB_PROGNOSE_STATUS     61716 non-null object


The dataset contains 26 columns which will be not of use to us. we will now discuss the meaning of these columns and translate them.
The rows correspond to an arrival and departure of a train from one station.

| Column |     Description     | New name/drop |
| :---: | :--- | :--- |
| BETRIEBSTAG | is the day the train departed but there is some inconsistencies as some trains departs after midnight in the day after |departure_day |
|  FAHRT_BEZEICHNER | is an unique indicator for a given train which doesn't seem that useful since LINIEN_ID does that also | Dropped |
|  BETREIBER_ID | is the id of the operator of the train which is always SBB in this dataset | Dropped |
|  BETREIBER_ABK | is the abrevation of the operator as above it is always SBB | Dropped |
|  BETREIBER_NAME | is the full name of the operator | Dropped |
|  PRODUKT_ID | is an id that tells us the type of transport it is. It's always trains apart from a few missing values that should also be trains | product_id |
|  LINIEN_ID | is an id for a given course | course_id |
|  LINIEN_TEXT | is the type of course | transport_type |
|  UMLAUF_ID | is an ID that tells us if there as been any changes in the programmation. It is always null | Dropped |
|  VERKEHRSMITTEL_TEXT | is just LINIEN_TEXT without the number at the end | Dropped |
|  ZUSATZFAHRT_TF | is a boolean that tells us if the train was not planned ahead | planned |
|  FAELLT_AUS_TF | is a boolean that tells us if the train was down | down |
|  BPUIC | is the service number | BPUIC |
|  HALTESTELLEN_NAME | is the name of the station | station_name |
|  ANKUNFTSZEIT | is the planned arrival time of the train at that station |planned_arrival_time|
|  AN_PROGNOSE | is the actual arrival time of the train | actual_arrival_time |
|  AN_PROGNOSE_STATUS | is an indicator of the deletion of the arrival of the train | cancelled_arrival |
|  ABFAHRTSZEIT | is the planned departure time of the train |planned_departure_time|
|  AB_PROGNOSE | is the actual departure time of the train |actual_departure_time|
|  AB_PROGNOSE_STATUS | is an indicator of the deletion of the departure of the train |cancelled_departure |
|  DURCHFAHRT_TF | don't know what it is but it is always false | Dropped |
|  ankunftsverspatung | is a boolean that tells us if the arrival of the train was delayed |arrival_delayed|
|  abfahrtsverspatung | is a boolean that tells us if the departure of the train was delayed |departure_delayed|
|  lod | is a link to a webpage that tells us more information about station | Dropped |
|  geopos | is the geoposition of the station | geopos |
|  GdeNummer | don't know | Dropped |

## Data preprocessing
We are going to drop some columns and rename the remaining ones.


In [5]:
df.drop(['lod','GdeNummer','DURCHFAHRT_TF','UMLAUF_ID','BETREIBER_NAME','BETREIBER_ABK','BETREIBER_ID','FAHRT_BEZEICHNER','VERKEHRSMITTEL_TEXT'],axis=1,inplace=True)
df.columns=['departure_day','product_id','course_id','transport_type','planned','down','BPUIC','station_name','planned_arrival_time',\
          'actual_arrival_time','cancelled_arrival','planned_departure_time','actual_departure_time','cancelled_departure','arrival_delayed','departure_delayed','geopos']
df.sample(5)

,departure_day,product_id,course_id,transport_type,planned,down,BPUIC,station_name,planned_arrival_time,actual_arrival_time,cancelled_arrival,planned_departure_time,actual_departure_time,cancelled_departure,arrival_delayed,departure_delayed,geopos
36753,2019-11-02,Zug,19577,S15,False,False,8503003,Zürich Stadelhofen,2019-11-02T20:11:00,2019-11-02T20:11:47,REAL,2019-11-02T20:12:00,2019-11-02T20:12:54,REAL,False,False,"47.3666105569,8.54846647022"
19843,2019-11-02,Zug,18578,S5,False,False,8503110,Rapperswil,2019-11-02T19:56:00,2019-11-02T19:56:42,REAL,2019-11-02T19:59:00,2019-11-02T19:59:35,REAL,False,False,"47.2248877672,8.81673057998"
12591,2019-11-02,Zug,25332,S30,False,False,8505406,Magadino-Vira,2019-11-02T11:44:00,2019-11-02T11:44:27,REAL,2019-11-02T11:46:00,2019-11-02T11:46:42,REAL,False,False,"46.1453223595,8.85017913795"
32497,2019-11-02,Zug,18879,S8,False,False,8503209,Pfäffikon SZ,2019-11-02T21:19:00,2019-11-02T21:20:49,REAL,NaN,NaN,PROGNOSE,False,False,"47.2029974467,8.77811068503"
11913,2019-11-02,Zug,24359,S3,False,False,8501200,Vevey,2019-11-02T16:52:00,2019-11-02T16:50:49,REAL,2019-11-02T16:52:00,2019-11-02T16:52:53,REAL,False,False,"46.4629974303,6.84345000982"


### Type conversion

We casted the date columns from string to datetime. We also considered to cast *geopos* but it's not required to use `geopy`.

In [6]:
df['planned_arrival_time'] =pd.to_datetime(df['planned_arrival_time'])
df['actual_arrival_time'] =pd.to_datetime(df['actual_arrival_time'])
df['planned_departure_time'] =pd.to_datetime(df['planned_departure_time'])
df['actual_departure_time'] =pd.to_datetime(df['actual_departure_time'])

In [7]:
#Translation of transport
df['product_id'].replace('Zug', 'train', inplace=True)

In [8]:
df.sample(4)

,departure_day,product_id,course_id,transport_type,planned,down,BPUIC,station_name,planned_arrival_time,actual_arrival_time,cancelled_arrival,planned_departure_time,actual_departure_time,cancelled_departure,arrival_delayed,departure_delayed,geopos
40656,2019-11-02,train,8656,S26,False,False,8500218,Olten,2019-11-02 15:18:00,2019-11-02 15:17:18,REAL,NaT,NaT,PROGNOSE,False,False,"47.3519295325,7.907688105"
10498,2019-11-02,train,30060,S3,True,False,8501201,La Tour-de-Peilz,2019-11-02 16:01:00,2019-11-02 16:04:20,REAL,2019-11-02 16:01:00,2019-11-02 16:04:51,REAL,True,True,"46.4554432937,6.85953018827"
43917,2019-11-02,train,1722,IR90,False,True,8501008,Genève,2019-11-02 16:30:00,NaT,PROGNOSE,2019-11-02 16:32:00,NaT,PROGNOSE,False,False,"46.2102022669,6.14245302875"
32987,2019-11-02,train,26074,R,False,False,8500293,Thalbrücke,2019-11-02 19:34:00,2019-11-02 19:34:00,PROGNOSE,2019-11-02 19:34:00,2019-11-02 19:34:24,PROGNOSE,False,False,"47.3085475984,7.68822003742"


In [14]:
df.loc[df['course_id'] == 1674].sort_values(by='planned_departure_time')

,departure_day,product_id,course_id,transport_type,planned,down,BPUIC,station_name,planned_arrival_time,actual_arrival_time,cancelled_arrival,planned_departure_time,actual_departure_time,cancelled_departure,arrival_delayed,departure_delayed,geopos
13402,2019-11-02,train,1674,IC51,False,False,8500010,Basel SBB,NaT,NaT,PROGNOSE,2019-11-02 13:58:00,2019-11-02 13:58:56,REAL,False,False,"47.5474041527,7.58955146721"
2697,2019-11-02,train,1674,IC51,False,False,8500113,Laufen,2019-11-02 14:15:00,2019-11-02 14:15:38,REAL,2019-11-02 14:21:00,2019-11-02 14:21:59,REAL,False,False,"47.4193383263,7.50263030739"
27550,2019-11-02,train,1674,IC51,False,False,8500109,Delémont,2019-11-02 14:37:00,2019-11-02 14:37:52,REAL,2019-11-02 14:42:00,2019-11-02 14:43:33,REAL,False,False,"47.3620006555,7.35005384248"
2698,2019-11-02,train,1674,IC51,False,False,8500105,Moutier,2019-11-02 14:51:00,2019-11-02 14:52:13,REAL,2019-11-02 14:52:00,2019-11-02 14:53:46,REAL,False,False,"47.2806661619,7.38103944266"
2699,2019-11-02,train,1674,IC51,False,False,8500159,Grenchen Nord,2019-11-02 15:00:00,2019-11-02 14:59:56,REAL,2019-11-02 15:01:00,2019-11-02 15:01:37,REAL,False,False,"47.1918033634,7.38946302189"
43338,2019-11-02,train,1674,IC51,False,False,8504300,Biel/Bienne,2019-11-02 15:10:00,2019-11-02 15:09:34,REAL,NaT,NaT,PROGNOSE,False,False,"47.1328920532,7.24290845288"


That shows it's normal to have NaT/Nan values for the date fields since the start point of the train has no arrival time and the arrival station has no planned departure.

In [35]:
df.groupby('cancelled_arrival').size()
#TODO translate this

cancelled_arrival
PROGNOSE    11351
REAL        50365
dtype: int64